In [61]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import collections 
import nltk
import re
from scipy.sparse import dok_matrix
import sklearn
%matplotlib inline

In [35]:
doc1 = "Game of Thrones Thrones is an amazing tv series!"
doc2 = "Game of Thrones is the best tv series!"
doc3 = "Game of Thrones is so great"

In [37]:
l_doc1 = re.sub(r"[^a-zA-Z0-9]", " ", doc1.lower()).split() #removing punctuations, smallcase the words and creating list of invidual words using split
l_doc1

['game', 'of', 'thrones', 'thrones', 'is', 'an', 'amazing', 'tv', 'series']

In [8]:
l_doc2 = re.sub(r"[^a-zA-Z0-9]", " ", doc1.lower()).split()
l_doc3 = re.sub(r"[^a-zA-Z0-9]", " ", doc1.lower()).split()

In [9]:
l_doc2
l_doc3

['game', 'of', 'thrones', 'is', 'an', 'amazing', 'tv', 'series']

In [54]:
def tokenize(line):
  return re.sub(r"[^a-zA-Z0-9]", " ", line.lower()).split()

def build_feature_map(X):
  word_types = set() #set whereever unique words is asked
  for datum in X:
    for word in tokenize(datum):
      word_types.add(word)
  # print(word_types)
  return {word : i for i , word in enumerate(word_types)}

def calculateBOW(wordset,l_doc):
  tf_diz = dict.fromkeys(wordset,0)
  print(tf_diz)
  for word in l_doc:
      tf_diz[word]=l_doc.count(word)
  return tf_diz

#another way of extracting features or calculating the count of each word is using scipy
#sparse matrix using dok_matrix 
def extract_features(word_to_idx , X):
  features = dok_matrix((len(X) , len(word_to_idx)))
  for i in range(len(X)):
    for word in tokenize(X[i]):
      if word in word_to_idx:
        features[i , word_to_idx[word]]+=1 
  return features


In [55]:
sample_data = [
    "Game of Thrones Thrones is an amazing tv series!",
    "Game of Thrones is the best tv series!",
    "Game of Thrones is so great"
]
wordset = build_feature_map(sample_data)
print(wordset)
# calculateBOW(wordset , l_doc1)
extract_features(wordset , sample_data)

{'thrones': 0, 'series': 1, 'is': 2, 'so': 3, 'great': 4, 'an': 5, 'best': 6, 'the': 7, 'game': 8, 'amazing': 9, 'tv': 10, 'of': 11}


<3x12 sparse matrix of type '<class 'numpy.float64'>'
	with 22 stored elements in Dictionary Of Keys format>

In [56]:
def read_XY_data(filename):
    X_data = []
    Y_data = []
    with open(filename, 'r') as f:
        for line in f:
            label, text = line.strip().split(' ||| ')
            X_data.append(text)
            Y_data.append(int(label))
    return X_data, Y_data

In [57]:
X_train ,Y_train = read_XY_data("train.txt")
X_test , Y_test = read_XY_data("test.txt")

In [59]:
#build feature map according to training data
wordset = build_feature_map(X_train)
X_train_vec = extract_features(wordset , X_train)
X_test_vec = extract_features(wordset, X_test)

In [65]:
clf = sklearn.linear_model.LogisticRegression(tol = 1e1)
clf.fit(X_train_vec, Y_train)
print (clf)

LogisticRegression(tol=10.0)


In [69]:
predictions = clf.predict(X_test_vec)
print(sklearn.metrics.classification_report(Y_test, predictions, target_names=['-1','0','1']))
print(sklearn.metrics.confusion_matrix(Y_test, predictions, labels=[-1,0, 1]))


              precision    recall  f1-score   support

          -1       0.68      0.69      0.69       912
           0       0.26      0.17      0.21       389
           1       0.69      0.78      0.73       909

    accuracy                           0.63      2210
   macro avg       0.54      0.55      0.54      2210
weighted avg       0.61      0.63      0.62      2210

[[629 116 167]
 [169  67 153]
 [125  79 705]]
